# Case 1 - Trees

### Table of Contents

1. **Importing Libraries**

2. **Loading Data**

3. **Random Forest**

4. **AdaBoosting**

5. **Ensemble of Elastic Net Predictions + AdaBoost Predictions**

## 1. Importing Libraries

In [42]:
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set() # Set searborn as default

from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

from sklearn.model_selection import RandomizedSearchCV

from sklearn.linear_model import LinearRegression

from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV

# Set seed for reproducibility
import random
random.seed(42)

## 2. Loading Data

In [43]:
# Loading the data into numpy arrays
X_train = np.loadtxt('../data/case1Data_Xtrain.csv', delimiter=',')
X_test = np.loadtxt('../data/case1Data_Xtest.csv', delimiter=',')
y_train = np.loadtxt('../data/case1Data_ytrain.csv', delimiter=',')
y_test = np.loadtxt('../data/case1Data_ytest.csv', delimiter=',')

### Summary of the data

This should be similar to the summary in case_1_data_wrangling.ipynb.

In [44]:
# Printing the shape of the data
print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)
print("y_train: ", y_train.shape)
print("y_test: ", y_test.shape)

# Size of the training and test data
n_train = X_train.shape[0]
n_test = X_test.shape[0]
p = X_train.shape[1]

# Printing the size of the training and test data
print("n_train: ", n_train) # number of training samples
print("n_test: ", n_test) # number of test samples
print("p: ", p) # number of features/variables/columns/parameters

# Checking for missing values in the wrangled data
missing_values_X_train = np.isnan(X_train)
print("Number of missing values in X_train: ", np.sum(missing_values_X_train))
missing_values_X_test = np.isnan(X_test)
print("Number of missing values in X_test: ", np.sum(missing_values_X_test))
missing_values_y_train = np.isnan(y_train)
print("Number of missing values in y_train: ", np.sum(missing_values_y_train))
missing_values_y_test = np.isnan(y_test)
print("Number of missing values in y_test: ", np.sum(missing_values_y_test))

X_train:  (80, 116)
X_test:  (20, 116)
y_train:  (80,)
y_test:  (20,)
n_train:  80
n_test:  20
p:  116
Number of missing values in X_train:  0
Number of missing values in X_test:  0
Number of missing values in y_train:  0
Number of missing values in y_test:  0


#### SELECTING FEATURES WITH BIC

In [45]:
def compute_bic(X, y, model):
    """Compute Bayesian Information Criterion (BIC) for a given model."""
    n, k = X.shape  # n = samples, k = features
    model.fit(X, y)
    residuals = y - model.predict(X)
    rss = np.sum(residuals**2)  # Residual sum of squares
    sigma2 = rss / n  # Estimated variance
    bic = n * np.log(sigma2) + k * np.log(n)  # BIC formula
    return bic

def stepwise_bic_selection(X, y):
    """Greedy forward selection based on BIC."""
    n_features = X.shape[1]
    selected_features = []  # Start with an empty set
    best_bic = np.inf
    model = LinearRegression()

    while True:
        bic_scores = []
        candidates = [i for i in range(n_features) if i not in selected_features]

        # Try adding each remaining feature
        for feature in candidates:
            current_features = selected_features + [feature]
            bic = compute_bic(X[:, current_features], y, model)
            bic_scores.append((feature, bic))

        # Select the feature that minimizes BIC
        bic_scores.sort(key=lambda x: x[1])
        best_new_feature, new_bic = bic_scores[0]

        # Stop if BIC does not improve
        if new_bic >= best_bic:
            break

        # Otherwise, update the best BIC and selected features
        best_bic = new_bic
        selected_features.append(best_new_feature)

    return selected_features, best_bic

# Run BIC feature selection
best_features, best_bic = stepwise_bic_selection(X_train, y_train)

print("Selected Feature Indices:", best_features)
print("Best BIC Score:", best_bic)

Selected Feature Indices: [31, 61, 35, 53, 67, 36, 81, 48, 9, 38, 77, 74, 104]
Best BIC Score: 492.75769252942996


#### SELECTING FEATURES WITH LASSO

In [46]:
lasso = LassoCV(cv=3, random_state=42) # LassoCV automatically performs cross-validation
lasso.fit(X_train, y_train) # Fit the model on the training data

# Select important features (nonzero coefficients)
selector = SelectFromModel(lasso, prefit=True) # Use the Lasso model to select features
selected_features_mask = selector.get_support()
selected_features = np.where(selected_features_mask)[0]  # Get indices of selected features

print("Selected Feature Indices:", selected_features)

Selected Feature Indices: [  6   8   9  10  12  27  28  29  31  34  35  40  42  44  48  50  53  54
  56  58  61  63  67  72  74  75  77  81  82  85 112]


# Models

## 3. Random Forest

In [47]:
# Initializing RandomForestRegressor
rf = RandomForestRegressor(n_estimators=100, max_features='sqrt', random_state=42)

# Performing cross-validation
cv_scores = cross_val_score(rf, X_train, y_train, cv=3, scoring='neg_mean_squared_error')

# Training the model
rf.fit(X_train, y_train)

# Making predictions
y_hat = rf.predict(X_test)

# Evaluate the model
rmse = np.sqrt(mean_squared_error(y_test, y_hat))
print(f'RMSE for random forest: {rmse:.4f}')

RMSE for random forest: 66.0736


#### WITH BIC FEATURES

In [48]:
# Initializing RandomForestRegressor
rf = RandomForestRegressor(n_estimators=100, max_features='sqrt', random_state=42)

# Performing cross-validation
cv_scores = cross_val_score(rf, X_train[:, best_features], y_train, cv=3, scoring='neg_mean_squared_error')

# Training the model
rf.fit(X_train[:, best_features], y_train)

# Making predictions
y_hat = rf.predict(X_test[:, best_features])

# Evaluate the model
rmse = np.sqrt(mean_squared_error(y_test, y_hat))
print(f'RMSE for random forest: {rmse:.4f}')

RMSE for random forest: 49.7670


#### WITH LASSO FEATURES

In [49]:
# Initializing RandomForestRegressor
rf = RandomForestRegressor(n_estimators=100, max_features='sqrt', random_state=42)

# Performing cross-validation
cv_scores = cross_val_score(rf, X_train[:, selected_features], y_train, cv=3, scoring='neg_mean_squared_error')

# Training the model
rf.fit(X_train[:, selected_features], y_train)

# Making predictions
y_hat = rf.predict(X_test[:, selected_features])

# Evaluate the model
rmse = np.sqrt(mean_squared_error(y_test, y_hat))
print(f'RMSE for random forest: {rmse:.4f}')

RMSE for random forest: 57.2986


## 4. AdaBoosting

In [50]:
# Define parameter distributions
param_dist = {
    'n_estimators': [10, 50, 100, 150, 200],
    'estimator__max_depth': [1, 3, 5, 10, 15],
    'learning_rate': np.logspace(-2, 1, 50),
}

# Create AdaBoostRegressor with DecisionTreeRegressor as base estimator
boost = AdaBoostRegressor(estimator=DecisionTreeRegressor())

# Use RandomizedSearchCV for efficiency
boost_search = RandomizedSearchCV(
    boost,
    param_distributions=param_dist,
    n_iter=300,  # Adjust based on computational resources
    cv=3,
    verbose=1,
    n_jobs=-1,
    scoring='neg_mean_squared_error',
    random_state=42
)

# Fit the model
boost_search.fit(X_train, y_train)

# Get best parameters and score
best_params = boost_search.best_params_
best_score = -boost_search.best_score_  # Converting back to positive MSE

# Use the best model to make predictions
y_hat = boost_search.predict(X_test)

# Evaluate the model
rmse = np.sqrt(mean_squared_error(y_test, y_hat))
print(f'Test RMSE for AdaBoost: {rmse:.4f}')


Fitting 3 folds for each of 300 candidates, totalling 900 fits
Test RMSE for AdaBoost: 49.2624


/opt/anaconda3/lib/python3.12/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


In [51]:
### AdaBoost only using the continuous features

# Define parameter distributions
param_dist = {
    'n_estimators': [10, 50, 100, 150, 200],
    'estimator__max_depth': [1, 3, 5, 10, 15],
    'learning_rate': np.logspace(-2, 1, 50),
}

# Create AdaBoostRegressor with DecisionTreeRegressor as base estimator
boost = AdaBoostRegressor(estimator=DecisionTreeRegressor())

# Use RandomizedSearchCV for efficiency
boost_search = RandomizedSearchCV(
    boost,
    param_distributions=param_dist,
    n_iter=10,  # Adjust based on computational resources
    cv=3,
    verbose=1,
    n_jobs=-1,
    scoring='neg_mean_squared_error',
    random_state=42
)

# Fit the model by only using the continuous features (first 95 columns)
boost_search.fit(X_train[:, :95], y_train)

# Get best parameters and score
best_params = boost_search.best_params_
best_score = -boost_search.best_score_  # Converting back to positive MSE

# Use the best model to make predictions
y_hat = boost_search.predict(X_test[:, :95])

# Evaluate the model
rmse = np.sqrt(mean_squared_error(y_test, y_hat))
print(f'Test RMSE for AdaBoost: {rmse:.4f}')


Fitting 3 folds for each of 10 candidates, totalling 30 fits
Test RMSE for AdaBoost: 51.8028


#### WITH BIC FEATURES

In [52]:
# Define parameter distributions
param_dist = {
    'n_estimators': [10, 50, 100, 150, 200],
    'estimator__max_depth': [1, 3, 5, 10, 15],
    'learning_rate': np.logspace(-2, 1, 50),
}

# Create AdaBoostRegressor with DecisionTreeRegressor as base estimator
boost = AdaBoostRegressor(estimator=DecisionTreeRegressor())

# Use RandomizedSearchCV for efficiency
boost_search = RandomizedSearchCV(
    boost,
    param_distributions=param_dist,
    n_iter=300,  # Adjust based on computational resources
    cv=3,
    verbose=1,
    n_jobs=-1,
    scoring='neg_mean_squared_error',
    random_state=42
)

# Fit the model
boost_search.fit(X_train[:, best_features], y_train)

# Get best parameters and score
best_params = boost_search.best_params_
best_score = -boost_search.best_score_  # Converting back to positive MSE

# Use the best model to make predictions
y_hat = boost_search.predict(X_test[:, best_features])

# Evaluate the model
rmse = np.sqrt(mean_squared_error(y_test, y_hat))
print(f'Test RMSE for AdaBoost: {rmse:.4f}')


Fitting 3 folds for each of 300 candidates, totalling 900 fits
Test RMSE for AdaBoost: 42.5136


#### WITH LASSO FEATURES

In [53]:
# Define parameter distributions
param_dist = {
    'n_estimators': [10, 50, 100, 150, 200],
    'estimator__max_depth': [1, 3, 5, 10, 15],
    'learning_rate': np.logspace(-2, 1, 50),
}

# Create AdaBoostRegressor with DecisionTreeRegressor as base estimator
boost = AdaBoostRegressor(estimator=DecisionTreeRegressor())

# Use RandomizedSearchCV for efficiency
boost_search = RandomizedSearchCV(
    boost,
    param_distributions=param_dist,
    n_iter=300,  # Adjust based on computational resources
    cv=3,
    verbose=1,
    n_jobs=-1,
    scoring='neg_mean_squared_error',
    random_state=42
)

# Fit the model
boost_search.fit(X_train[:, selected_features], y_train)

# Get best parameters and score
best_params = boost_search.best_params_
best_score = -boost_search.best_score_  # Converting back to positive MSE

# Use the best model to make predictions
y_hat = boost_search.predict(X_test[:, selected_features])

# Evaluate the model
rmse = np.sqrt(mean_squared_error(y_test, y_hat))
print(f'Test RMSE for AdaBoost: {rmse:.4f}')

Fitting 3 folds for each of 300 candidates, totalling 900 fits
Test RMSE for AdaBoost: 45.7866


/opt/anaconda3/lib/python3.12/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


## Regression Tree

In [54]:
# create a decisiontreeregressor/classifier
dtree = DecisionTreeRegressor()

# Fit the tree regressor/classifier
dtree.fit(X_train, y_train)

# Predict the target variable
y_pred = dtree.predict(X_test)

# Calculating the root mean squared error (RMSE)
mse = mean_squared_error(y_test, y_pred)
print(f'RMSE: {rmse}')

RMSE: 45.78660673398


#### WITH BIC FEATURES

In [55]:
# create a decisiontreeregressor/classifier
dtree = DecisionTreeRegressor()

# Fit the tree regressor/classifier
dtree.fit(X_train[:, best_features], y_train)

# Predict the target variable
y_pred = dtree.predict(X_test[:, best_features])

# Calculating the root mean squared error (RMSE)
mse = mean_squared_error(y_test, y_pred)
print(f'RMSE: {rmse}')

RMSE: 45.78660673398


#### WITH LASSO FEATURES

In [56]:
# create a decisiontreeregressor/classifier
dtree = DecisionTreeRegressor()

# Fit the tree regressor/classifier
dtree.fit(X_train[:, selected_features], y_train)

# Predict the target variable
y_pred = dtree.predict(X_test[:, selected_features])

# Calculating the root mean squared error (RMSE)
mse = mean_squared_error(y_test, y_pred)
print(f'RMSE: {rmse}')

RMSE: 45.78660673398


## 5. Ensemble of Elastic Net Predictions + AdaBoost Predictions

### Elastic Net

In [66]:
# Load libraries
from sklearn.linear_model import ElasticNetCV
import warnings

# Setting a range of alphas to test
alphas = np.logspace(-4, 0, 100)

# Setting a range of l1_ratios
l1_ratios = np.logspace(-10, 0, 100)

with warnings.catch_warnings(): # done to disable all the convergence warnings from elastic net
    warnings.simplefilter("ignore")
    
    # Fitting the Elastic Net model on the training data
    model = ElasticNetCV(cv=3, l1_ratio = l1_ratios, alphas=alphas, fit_intercept=False).fit(X_train, y_train)

# Printing the optimal alpha
print(f'Optimal alpha: {model.alpha_}\n')
print('For l1_ratio = 0 the penalty is an L2 penalty. For l1_ratio = 1 it is an L1 penalty. For 0 < l1_ratio < 1, the penalty is a combination of L1 and L2.')
print(f'Optimal l1_ratio: {model.l1_ratio_}')

# Using the optimal lambda from the ElasticNetCV model to predict the target values on the test data
y_hat_elastic = model.predict(X_test)

# Calculating the RMSE of the ElasticNetCV model
rmse = np.sqrt(mean_squared_error(y_test, y_hat_elastic))

# Printing the RMSE
print('Root MSE from OLS with elastic net regression and cross validation to find optimal lambda and model parameters:')
print(f'RMSE: {rmse}')

Optimal alpha: 0.9111627561154896

For l1_ratio = 0 the penalty is an L2 penalty. For l1_ratio = 1 it is an L1 penalty. For 0 < l1_ratio < 1, the penalty is a combination of L1 and L2.
Optimal l1_ratio: 1.0
Root MSE from OLS with elastic net regression and cross validation to find optimal lambda and model parameters:
RMSE: 26.094967036910088


### AdaBoost

In [67]:
# Define parameter distributions
param_dist = {
    'n_estimators': [10, 50, 100, 150, 200],
    'estimator__max_depth': [1, 3, 5, 10, 15],
    'learning_rate': np.logspace(-2, 1, 50),
}

# Create AdaBoostRegressor with DecisionTreeRegressor as base estimator
boost = AdaBoostRegressor(estimator=DecisionTreeRegressor())

# Use RandomizedSearchCV for efficiency
boost_search = RandomizedSearchCV(
    boost,
    param_distributions=param_dist,
    n_iter=300,  # Adjust based on computational resources
    cv=3,
    verbose=1,
    n_jobs=-1,
    scoring='neg_mean_squared_error',
    random_state=42
)

# Fit the model
boost_search.fit(X_train, y_train)

# Get best parameters and score
best_params = boost_search.best_params_
best_score = -boost_search.best_score_  # Converting back to positive MSE

# Use the best model to make predictions
y_hat = boost_search.predict(X_test)

# Evaluate the model
rmse = np.sqrt(mean_squared_error(y_test, y_hat))
print(f'Test RMSE for AdaBoost: {rmse:.4f}')

Fitting 3 folds for each of 300 candidates, totalling 900 fits
Test RMSE for AdaBoost: 55.8461


### Average

In [68]:
# Performing an average of the predictions from the ElasticNetCV and AdaBoost models
y_hat_weighted = (y_hat_elastic + y_hat_boost) / 2

# Calculating the RMSE of the weighted average model
rmse = np.sqrt(mean_squared_error(y_test, y_hat_weighted))

# Printing the RMSE
print('Root MSE from weighted average of ElasticNetCV and AdaBoost models:')
print(f'RMSE: {rmse}')

Root MSE from weighted average of ElasticNetCV and AdaBoost models:
RMSE: 39.37167178503088
